## Web3.py

In [278]:
import json
import web3

from web3 import Web3, TestRPCProvider, IPCProvider
from solc import compile_source
from web3.contract import ConciseContract
import web3.eth as eth

Web3 provides a few providers to connect to an ethereum node through:
- HTTP based JSON-RPC server
- IPC socket based JSON-RPC server
- WSS based JSON-RPC server

In [279]:
w3 = Web3(IPCProvider('/path/to/geth.ipc'))

In [280]:
w3.eth.blockNumber

3

### Geth-style Proof of Authority
This middleware is required to connect to `geth --dev` or the Rinkeby public network.

For example, to connect to a local `geth --dev` instance on Linux:

#### Why is `geth_poa_middleware` necessary?
There is no strong community consensus on a single Proof-of-Authority (PoA) standard yet. Some nodes have successful experiments running, though. One is go-ethereum (geth), which uses a prototype PoA for it’s development mode and the Rinkeby test network.

Unfortunately, it does deviate from the yellow paper specification, which constrains the `extraData` field in each block to a maximum of 32-bytes. Geth’s PoA uses more than 32 bytes, so this middleware modifies the block data a bit before returning it.


In [281]:
from web3.middleware import geth_poa_middleware
w3.middleware_stack.inject(geth_poa_middleware, layer=0)
w3.version.node

'Geth/v1.8.3-stable/darwin-amd64/go1.10.1'

#### List all transactions

The only way to list all transactions in the blockchain, is to iterate over each block and the transactions inside the block.

In [282]:
def list_all_transactions():
    n = w3.eth.blockNumber;
    txs = list();
    for i in range(n+1):
        block = w3.eth.getBlock(i, True)
        for j in range(len(block.transactions)):
            if block.transactions[j]['from'] == w3.eth.accounts[0]:
                txs.append([block.transactions[j]])
    print(f"Blocks: {n}")
    print(f"Transactions: {len(txs)}")
    return txs

In [283]:
w3.eth.enable_unaudited_features()
# w3.eth.account
w3.eth.accounts

['0x480B60c8c84Ea3793394C4317f8f10fd26A0f66F']

### Solidity

In [284]:
# Solidity source code
contract_source_code = '''
pragma solidity ^0.4.0;

contract Greeter {
    string public greeting;

    function Greeter() {
        greeting = 'Hello';
    }

    function setGreeting(string _greeting) public {
        greeting = _greeting;
    }

    function greet() constant returns (string) {
        return greeting;
    }
}
'''

In [285]:
compiled_sol = compile_source(contract_source_code) # Compiled source code
contract_interface = compiled_sol['<stdin>:Greeter']

In [286]:
contract_interface['bin']

'6060604052341561000f57600080fd5b6040805190810160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506000908051906020019061005a929190610060565b50610105565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106100a157805160ff19168380011785556100cf565b828001600101855582156100cf579182015b828111156100ce5782518255916020019190600101906100b3565b5b5090506100dc91906100e0565b5090565b61010291905b808211156100fe5760008160009055506001016100e6565b5090565b90565b61041a806101146000396000f300606060405260043610610057576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063a41368621461005c578063cfae3217146100b9578063ef690cc014610147575b600080fd5b341561006757600080fd5b6100b7600480803590602001908201803590602001908080601f016020809104026020016040519081016040528093929190818152602001838380828437820191505050505050919050506101d5565b005b34156100c457600080fd5b6100cc6101ef565b6040518080602001828103825283818151815

In [287]:
contract = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

In [288]:
contract_interface['abi']

[{'constant': False,
  'inputs': [{'name': '_greeting', 'type': 'string'}],
  'name': 'setGreeting',
  'outputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greet',
  'outputs': [{'name': '', 'type': 'string'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greeting',
  'outputs': [{'name': '', 'type': 'string'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'constructor'}]

**Deploy Contract** and save hash 

_DeprecationWarning: deploy is deprecated in favor of_ `contract.constructor.transact`

In [ ]:
tx_hash = contract.deploy(transaction={'from': w3.eth.accounts[0], 'gas':410000})

In [290]:
tx_hash

HexBytes('0x0428f1bb713541546d700f6f6a0b003ec6ab70fe8cc45f0e85d975be69e458d1')

**# NOTE: Wait ~1sec before calling `getTransactionReceipt`! The blockchain needs a little bit of time to deploy the contract and finalize the transaction!**

In [291]:
tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
contract_address = tx_receipt['contractAddress']

In [292]:
contract_instance = w3.eth.contract(abi=contract_interface['abi'], address=contract_address, ContractFactoryClass=ConciseContract)

In [294]:
print(contract_instance.greet())

Hello


In [295]:
contract_instance.setGreeting('Nihao', transact={'from': w3.eth.accounts[0]})

HexBytes('0x0f4a08656b9c59dd99a4adb98bf97ca26720fd82b7b80b72c829f1d954d9ee02')

In [16]:
print('Setting value to: Nihao')
print('Contract value: {}'.format(contract_instance.greet()))

Setting value to: Nihao
Contract value: Nihao


In [ ]:
txs = list_all_transactions()

In [ ]:
txs

In [10]:
block = w3.eth.getBlock(0, True)
block.transactions

[]

In [13]:
account = w3.eth.accounts[0]

In [18]:
w3.eth.getBlock(1, True).transactions[0]['from']

AttributeError: 'NoneType' object has no attribute 'keys'

## Crowdfunding

In [241]:
import serpent
import re

In [258]:
CONTRACT = "./crowdfunding/crowdfunding.se"
program = open(CONTRACT).read()
machine_code = serpent.compile(program)
abi_signature = serpent.mk_full_signature(program)

**NB: The `serpent.mk_full_signature()` function will produce an ABI not accepted by web3. We need to strip the trailing `()` in the function names to make it work!** Go figure.. :) 

In [259]:
for sig in abi_signature:
    sig['name'] = re.sub(r'\(.*\)', '', sig['name'])
#     sig['name'] = sig['name'].rstrip("()")

In [260]:
abi_signature

[{'name': 'add',
  'type': 'function',
  'constant': False,
  'inputs': [{'name': 'a', 'type': 'int256'}, {'name': 'b', 'type': 'int256'}],
  'outputs': [{'name': 'out', 'type': 'int256'}]},
 {'name': 'add_contrib',
  'type': 'function',
  'constant': False,
  'inputs': [],
  'outputs': []},
 {'name': 'clear',
  'type': 'function',
  'constant': False,
  'inputs': [],
  'outputs': []},
 {'name': 'contribute',
  'type': 'function',
  'constant': False,
  'inputs': [],
  'outputs': [{'name': 'out', 'type': 'int256'}]},
 {'name': 'create_campaign',
  'type': 'function',
  'constant': False,
  'inputs': [{'name': 'recipient', 'type': 'bytes'},
   {'name': 'goal', 'type': 'int256'},
   {'name': 'timelimit', 'type': 'int256'}],
  'outputs': [{'name': 'out', 'type': 'int256'}]},
 {'name': 'get_contrib_total',
  'type': 'function',
  'constant': False,
  'inputs': [],
  'outputs': [{'name': 'out', 'type': 'int256'}]},
 {'name': 'get_goal',
  'type': 'function',
  'constant': False,
  'inputs':

In [261]:
contract = w3.eth.contract(abi=abi_signature, bytecode=machine_code)

In [262]:
w3.eth.accounts

['0xA15925Cb50f2735d329d7FCC350Af671eDA8d909',
 '0x48024053A6b119fdc7A3B8119aeE4dE5Ef56Dcff',
 '0xfAD99564faAD56D1238db24357e05D4796Ceaff8']

In [263]:
tx_hash = contract.deploy(transaction={'from': w3.eth.accounts[0], 'gas':410000})

/Users/StefanoFiora/.virtualenvs/blockchain/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: deploy is deprecated in favor of contract.constructor.transact
  """Entry point for launching an IPython kernel.


In [264]:
tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
contract_address = tx_receipt['contractAddress']
contract_instance = w3.eth.contract(abi=abi_signature, address=contract_address, ContractFactoryClass=ConciseContract)

In [265]:
contract_instance

In [266]:
list_all_transactions()[1]

Blocks: 37
Transactions: 35


[AttributeDict({'blockHash': HexBytes('0xfd81b00e5d522e68c96d3a651ccacc586ff8a6e292fc0fecd14a0c801ee95b08'),
  'blockNumber': 2,
  'from': '0xA15925Cb50f2735d329d7FCC350Af671eDA8d909',
  'gas': 121514,
  'gasPrice': 1,
  'hash': HexBytes('0xd5b41932bcef3bdee01695015975628652b7685bd85405e2c79f3c65986961e9'),
  'input': '0x2212dbc3',
  'nonce': 1,
  'to': '0xE17CF4b3583253a6C636e0D5680eb3dD91dc3C8F',
  'transactionIndex': 0,
  'value': 0,
  'v': 2710,
  'r': HexBytes('0xdd0b0e0613477338ac3e52770eca1d738dfbe9da244d033094c7f028b60fcc80'),
  's': HexBytes('0x51c4014c449cd189edd65185af5ec9072dcc6f3c71a111c02e822a7aa47f9fbd')})]

**NOTE: If you add `transact` to the function call, the call will result in an actual transactiona added to the blockchain. Otherwise, the call will be just a 'simulated' stateless call, any modification to the contract and its storage will not be persistent.**

In [267]:
contract_instance.get_contrib_total()

0

In [268]:
contract_instance.add(3, 4, transact={'from':w3.eth.accounts[0]})

HexBytes('0xee9d7652d511ade11b9f38fe5e36b85a30101fcb382962110982ad0049020de7')

In [269]:
# contract_instance.add_contrib(2, transact={'from':w3.eth.accounts[0]})

In [273]:
str.encode(w3.eth.accounts[0])

b'0xA15925Cb50f2735d329d7FCC350Af671eDA8d909'

In [274]:
contract_instance.create_campaign(str.encode(w3.eth.accounts[0]), w3.toWei("10.0", "ether"), 10 * 24 * 3600, transact={'from':w3.eth.accounts[0]})

HexBytes('0x5df93e42c5f08c61f41806d320313013d2ae8c254235f16ee3ce45f06741f7ba')

In [207]:
contract_instance.add_contrib(transact={'from':w3.eth.accounts[0], 'value': w3.toWei("2.0", "ether")})

HexBytes('0x4a17d6b88dc1619c72e3a60a32793e7dd8caf3cf3dafe25d2b1de2bc868d73ef')

In [208]:
w3.eth.getBalance(contract_address)

11000000000000000000

**Create some accounts and go through a crowdfunding campaign**

In [ ]:
w3.personal.newAccount('test')

In [169]:
w3.eth.accounts

['0xA15925Cb50f2735d329d7FCC350Af671eDA8d909',
 '0x48024053A6b119fdc7A3B8119aeE4dE5Ef56Dcff',
 '0xfAD99564faAD56D1238db24357e05D4796Ceaff8']

In [200]:
w3.personal.unlockAccount(w3.eth.accounts[1], passphrase="test")
w3.personal.unlockAccount(w3.eth.accounts[2], passphrase="test")

True

In [189]:
# send some ether to new accounts
w3.eth.sendTransaction({"to":w3.eth.accounts[2], "from":w3.eth.accounts[0], "value":w3.toWei("100", "ether")})

HexBytes('0x9ffe9db2445290f2990e830819e47f0c9033b934fbe61405bfbd0c078941d16d')

In [209]:
for i in range(len(w3.eth.accounts)):
    print(f"Account {i} balance: {w3.eth.getBalance(w3.eth.accounts[i])}")

Account 0 balance: 115792089237316195423570985008687907853269984665640564039128084007913129693426
Account 1 balance: 95499999999999973248
Account 2 balance: 196999999999999973248


In [ ]:
data recipient
data goal
data deadline
data contrib_total
data contrib_count
data contribs[](sender, value)   # define infinite array

def create_campaign(recipient:str, goal, timelimit):
    # campaign already exists
    if self.recipient:
        return("Already initialized")
    self.recipient = recipient
    self.goal = goal
    self.deadline = block.timestamp + timelimit
    return self.recipient

def contribute():
    # Update contribution total
    total_contributed = self.contrib_total + msg.value
    self.contrib_total = total_contributed

    # Record new contribution
    sub_index = self.contrib_count
    self.contribs[sub_index].sender = msg.sender
    self.contribs[sub_index].value = msg.value
    self.contrib_count = sub_index + 1
    
    # refund if expired or goal reached
    self.refund()
    return self.contrib_total

def refund():
    # If expired, refund all contributors
    if block.timestamp > self.deadline or
        total_contributed >= self.goal::
        i = 0
        c = self.contrib_count
        while i < c:
            send(self.contribs[i].sender, 
                 self.contribs[i].value)
            i += 1
        self.clear()
    return(2)

def progress_report():
    return(self.contrib_total)

def clear():
    if self == msg.sender:
        self.recipient = 0
        self.goal = 0
        self.deadline = 0
        c = self.contrib_count
        self.contrib_count = 0
        self.contrib_total = 0
        i = 0
        while i < c:
            self.contribs[i].sender = 0
            self.contribs[i].value = 0
            i += 1

## References

- https://hackernoon.com/ethereum-smart-contracts-in-python-a-comprehensive-ish-guide-771b03990988
- https://medium.com/@ConsenSys/a-101-noob-intro-to-programming-smart-contracts-on-ethereum-695d15c1dab4